In [1]:
import pandas as pd
import numpy as np
import re
from completing_missing_clinician_questionnaire_2021 import map_additional_column_name_to_2021_clinician_column_name, remove_from_stu_columns, map_additional_column_name_to_2021_student_column_name, IsClinician

from creating_the_clinic_dataset.step2_merge_2021_2022.utils import fix_wrong_record_id, fill_id, remove_test_id

ModuleNotFoundError: No module named 'creating_the_clinic_dataset'

In [2]:
imputation_data = pd.read_csv(r"../original_data/Student_Clinician_data_2021.csv", na_values= ' ')
# the warning rooted from the nan values


rename = {'cssrs_followup_timestamp':'cssrs_fw_maya_timestamp', 'cssrs_followup_complete':'cssrs_fw_maya_complete'}
additional_data = additional_data.rename(rename, axis=1)

## error record_id = 363
additional_data = additional_data[additional_data['record_id']!=363]

#do the imputation id like of 2022 dataset
additional_data = fill_id(additional_data)
df = remove_test_id(df)

C:\Users\nogur\AppData\Local\Temp\ipykernel_7348\915307582.py:1: DtypeWarning: Columns (48) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2021 = pd.read_csv(r"../original_data/Schneider Depression Clinic Database.csv", na_values= ' ')
C:\Users\nogur\AppData\Local\Temp\ipykernel_7348\915307582.py:2: DtypeWarning: Columns (44,46,51,53,99,325,366,439,453,459,464,466,501,507,512,514,519,521,530,532,569,571,580,599,766,771,773,779,784,789,794,798,800,804,809,811,813,816,818,827,829,849,866,868,877,896,906,941,964,1002,1014,1025,1063,1068,1070,1076,1081,1086,1091,1093,1095,1097,1099,1100,1126,1142,1149,1165,1168,1172,1173,1177,1178,1179,1180,1182,1183,1184,1187,1188,1190,1207,1212,1229,1232,1260,1262,1264,1266,1268,1271,1281,1283,1285,1291,1293,1297,1305,1306,1319,1321,1324,1326,1338,1340,1342,1343,1344,1350,1351,1352,1353,1465,1467,1468,1470,1486,1489,1492,1496,1513,1515,1516,1517,1523,1524,1525,1526,1580,1650,1653,1654) have mixed types. Specify dtype optio

In [ ]:
def upload_to_sql(connection_str, column, row, imputation_row):
    
    
    if row.isna()[column]:
    
        try:
            questionnaire_alias = row.quesionairre_alias[column]
        except KeyError:
            return
        
        
        imputation_value = imputation_row[questionnaire_alias]

        
        conn = psycopg2.connect(connection_str)
        cur = conn.cursor()
        cur.execute("""UPDATE  table
        SET {0} = '{1}'
        WHERE primary_key = "{2}";""".format(column, imputation_value, imputation_row['primary_key']))

        conn.commit()
        cur.close()
        conn.close()


In [ ]:
def fill_columns (connection_str, df, imputation_df):
    
    for _, row in df.iterrows():
        for column in df.columns:
            
            imputation_row = imputation_df[imputation_df['primary_key'] == row['primary_key']]
            
            upload_to_sql(connection_str, column, row, imputation_row)
            



In [ ]:
def impute_data(data_df, impute_df, column_aliases, connection_str, table_name):
    data_df.set_index('primary_key', in_place=True)
    impute_df.rename(column_aliases, axis=1, in_place=True)
    # fill the missing values with the imputation values
    data_df = data_df.fillna(impute_df.set_index('primary_key'))

    # upload the dataframe to the database
    with psycopg2.connect(connection_str) as conn:
        data_df.to_sql(table_name, conn, if_exists='replace', index_label='primary_key')

In [7]:
def fill_columns(columns_to_add, map_for_data, df_2021, row):
    for column in columns_to_add:
        if not row.isna()[column]:
            column_rename = map_for_data[column]

            df_2021.loc[(df_2021['id']==row['id']) & (df_2021['redcap_event_name']==row['redcap_event_name']), column_rename] = row[column]
    return df_2021



# # check if not equals 
# if not (df_2021[(df_2021['id']==row['id']) & (df_2021['redcap_event_name']==row['redcap_event_name'])][column_rename] == row[column]).all():
#     print(column)
#change value


In [8]:
#         only_clin_cols = [column for column in clinician_columns_add if column not in stu_columns_add]
#         print(row[only_clin_cols].isna().all())
        

In [9]:

# cols_2022 = list(df_2022.columns)
# cols_add = list(additional_data.columns)

In [10]:
for i, row in imputation_data.iterrows():
    
    fill_columns(chameleon_columns_add, map_chameleon_2021_add, df_2021, row)
    
    if IsClinician(row):

        fill_columns(stu_columns_add, map_for_student_data, df_2021, row)
    
    else:
        fill_columns(clinician_columns_add, map_for_clinician_data, df_2021, row)


In [11]:
df_2021.to_csv(r'../preprocessed_data/2021_data_imputed_with_clin_stu.csv', index=False)
df_2022.to_csv(r'../preprocessed_data/2022_data_imputed_with_clin_stu.csv', index=False)